# UCR Data

In [8]:
include("generate_artifacts.jl")
data_dir = "../data"
artifact_name = "ucr";

### Retrieve data

In [9]:
using PyCall
using BSON: @save
run = true
if run 
    py"""
    from tslearn.datasets import UCR_UEA_datasets

    X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset('ECG200')
    """
    
    # Relabel: 
    y_train = ifelse.(py"y_train".==-1,0,1)
    y_test = ifelse.(py"y_test".==-1,0,1)


    data = Dict(
        "X_train" => py"X_train",
        "y_train" => y_train,
        "X_test" => py"X_test",
        "y_test" => y_test
    )

    @save joinpath(data_dir, artifact_name * "_data.bson") data
end

/Users/FA31DU/.julia/conda/3/lib/python3.9/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [10]:
using BSON: @load
@load joinpath(data_dir, artifact_name * "_data.bson") data
y_train, X_train, X_test, y_test = values(data)
n_train = size(X_train)[1]
input_dim = size(X_train)[2]
output_dim = 1;

### Classifier

#### MLP

In [11]:
using Flux
# MLP
function mlp(;input_dim, hidden_dim, output_dim, activation=Flux.relu)
    model = Chain(
        Dense(input_dim, hidden_dim, activation),
        Dropout(0.1),
        Dense(hidden_dim, output_dim)
    )  
    return model
end

mlp (generic function with 1 method)

In [12]:
using Flux: DataLoader
kw_args = (input_dim=input_dim,hidden_dim=128,output_dim=output_dim)
model = mlp(;kw_args...)
loss(x, y) = Flux.Losses.logitbinarycrossentropy(model(x), y)
bs = Int(round(n_train/10))
data = DataLoader((X_train[:,:,1]',Flux.unsqueeze(y_train,1)),batchsize=bs)
test_data = zip(Flux.unstack(X_test[:,:,1],1),Flux.unsqueeze(y_test,1));

In [15]:
using Flux.Optimise: update!, ADAM
using Statistics
opt = ADAM()
epochs = 20
avg_loss(data) = mean(map(d -> loss(d[1],d[2]), data))
accuracy(data) = mean(map(d -> mean(round.(Flux.σ.(model(d[1]))) .== d[2]), data))

if true
  for epoch = 1:epochs
    for d in data
      gs = gradient(params(model)) do
        l = loss(d...)
      end
      update!(opt, params(model), gs)
    end
    println("Epoch " * string(epoch))
    @show avg_loss(data)
    @show accuracy(data)
    @show accuracy(test_data)
  end
  @save joinpath(data_dir, artifact_name * "_model.bson") model
end

Epoch 1


avg_loss(data) = 0.43564407463113186
accuracy(data) = 

0.7699999999999999


accuracy(test_data) = 0.79
Epoch 2
avg_loss(data) = 0.34004401632163284
accuracy(data) = 0.85
accuracy(test_data) = 0.8
Epoch 3
avg_loss(data) = 0.3028721270873455
accuracy(data) = 0.86
accuracy(test_data) = 0.81
Epoch 4
avg_loss(data) = 0.27314380899935975
accuracy(data) = 0.8600000000000001
accuracy(test_data) = 0.84
Epoch 5
avg_loss(data) = 0.25103696045054885
accuracy(data) = 0.8800000000000001
accuracy(test_data) = 0.84
Epoch 6
avg_loss(data) = 0.2334818298950181
accuracy(data) = 0.9000000000000001
accuracy(test_data) = 0.85
Epoch 7
avg_loss(data) = 0.21950813887769596
accuracy(data) = 0.9200000000000002
accuracy(test_data) = 0.86
Epoch 8
avg_loss(data) = 0.20873821320936078
accuracy(data) = 0.9200000000000002
accuracy(test_data) = 0.85
Epoch 9
avg_loss(data) = 0.19853346934855418
accuracy(data) = 0.93
accuracy(test_data) = 0.85
Epoch 10
avg_loss(data) = 0.18743479852722733
accuracy(data) = 0.9400000000000002
accuracy(test_data) = 0.85
Epoch 11
avg_loss(data) = 0.17871306372098092

0.17007527346369813
accuracy(data) = 0.95
accuracy(test_data) = 0.85
Epoch 13
avg_loss(data) = 0.16173779092380702
accuracy(data) = 0.95
accuracy(test_data) = 0.85
Epoch 14
avg_loss(data) = 0.15473807646148968
accuracy(data) = 0.95
accuracy(test_data) = 0.85
Epoch 15
avg_loss(data) = 0.14955395759877854
accuracy(data) = 0.95
accuracy(test_data) = 0.87
Epoch 16
avg_loss(data) = 0.14219930797140307
accuracy(data) = 0.95
accuracy(test_data) = 0.84
Epoch 17
avg_loss(data) = 0.1365292525088101
accuracy(data) = 0.95
accuracy(test_data) = 0.86
Epoch 18
avg_loss(data) = 0.1304868182398107
accuracy(data) = 0.9600000000000002
accuracy(test_data) = 0.85
Epoch 19
avg_loss(data) = 0.12487738929514436
accuracy(data) = 0.9600000000000002
accuracy(test_data) = 0.86
Epoch 20
avg_loss(data) = 0.1198903205060956
accuracy(data) = 0.9700000000000001
accuracy(test_data) = 0.85


### Generate artifact

In [16]:
datafiles = [artifact_name * "_data.bson",artifact_name * "_model.bson"]
generate_artifact(datafiles)

┌ Warning: For automatic github deployment, need GITHUB_TOKEN. Not found in ENV, attemptimg global git config.
└ @ Main /Users/FA31DU/Documents/code/CounterfactualExplanations.jl/dev/artifacts/generate_artifacts.jl:13


┌ Info: Binding ucr_data in Artifacts.toml...
└ @ Main /Users/FA31DU/Documents/code/CounterfactualExplanations.jl/dev/artifacts/generate_artifacts.jl:56


┌ Info: Binding ucr_model in Artifacts.toml...
└ @ Main /Users/FA31DU/Documents/code/CounterfactualExplanations.jl/dev/artifacts/generate_artifacts.jl:56
┌ Info: Uploading tarballs to pat-alt/CounterfactualExplanations.jl tag `data`
└ @ Main /Users/FA31DU/Documents/code/CounterfactualExplanations.jl/dev/artifacts/generate_artifacts.jl:67


--> Deleting: ucr_model.tar.gz
--> Deleting: ucr_data.tar.gz
--> Uploading: ucr_model.tar.gz
--> Uploading: ucr_data.tar.gz


┌ Info: Artifacts.toml file now contains all bound artifact names
└ @ Main /Users/FA31DU/Documents/code/CounterfactualExplanations.jl/dev/artifacts/generate_artifacts.jl:73
